In [1]:
%pip install --user ragas langchain-openai pandas datasets

In [2]:
# Imports and Model Configuration (Ragas)
import json
import pandas as pd
import os
import numpy as np
from datasets import Dataset
import sys
import traceback

# Ragas specific imports
from ragas import evaluate
from ragas.metrics import (
    # Independent of Ground Truth (Reference-Free)
    context_precision,      
    answer_relevancy,       
    faithfulness,           
    
    # Dependent on Ground Truth (Reference-Required)
    context_recall,         
    context_entity_recall,  
    answer_correctness,     
    answer_similarity       
)

# Ragas 1.0+ uses Langchain models
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings 

from ragas.exceptions import RagasException

In [ ]:
# --- AZURE CONFIGURATION (!!! REPLACE PLACEHOLDERS BELOW !!!) ---
# ⚠️ IMPORTANT: Replace with your actual key
AZURE_API_KEY = "<YOUR_AZURE_OPENAI" 
AZURE_ENDPOINT = "https://<YOUR_AZURE_OPENAI_ENDPOINT>.openai.azure.com/"
AZURE_CHAT_DEPLOYMENT = "gpt-4o-08-06" # Model used for evaluation logic (e.g., faithfulness, relevancy)
AZURE_EMBEDDINGS_DEPLOYMENT = "text_embedding_ada_002" # Model used for semantic metrics (e.g., similarity)
AZURE_API_VERSION = "2024-12-01-preview" 
# -----------------------------------------------------------------

# Set environment variable for Langchain/Ragas to pick up the key
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_API_KEY

In [4]:
# Initialize Ragas LLM (for the main evaluation)
try:
    ragas_llm = AzureChatOpenAI(
        api_key=AZURE_API_KEY, 
        azure_endpoint=AZURE_ENDPOINT,
        deployment_name=AZURE_CHAT_DEPLOYMENT, 
        api_version=AZURE_API_VERSION,
        temperature=0.0
    )
    print("✅ Ragas Chat LLM initialized successfully!")
    
except Exception as e:
    print(f"❌ Error initializing Ragas AzureChatOpenAI LLM: {e}")
    sys.exit(1)

# Initialize Ragas Embeddings LLM
try:
    ragas_embeddings_llm = AzureOpenAIEmbeddings(
        api_key=AZURE_API_KEY,
        azure_endpoint=AZURE_ENDPOINT,
        deployment=AZURE_EMBEDDINGS_DEPLOYMENT, 
        openai_api_version=AZURE_API_VERSION,
        model=AZURE_EMBEDDINGS_DEPLOYMENT
    )
    print("✅ Ragas Embeddings LLM initialized successfully!")
    
except Exception as e:
    print(f"❌ Error initializing Ragas AzureOpenAIEmbeddings LLM: {e}")
    sys.exit(1)

✅ Ragas Chat LLM initialized successfully!
✅ Ragas Embeddings LLM initialized successfully!


In [5]:
# Metrics to run (7 Core Functional Metrics)
REQUESTED_METRICS = [
    # Reference-Free Metrics (Independent of Ground Truth)
    faithfulness,         # Measures how factually accurate the generated answer is based on the context.
    answer_relevancy,     # Measures how relevant the generated answer is to the question.
    context_precision,    # Measures how relevant the retrieved context chunks are to answering the question.
    
    # Reference-Required Metrics (Dependent on Ground Truth)
    context_recall,       # Measures the ability of the retrieval system to surface all necessary information.
    context_entity_recall,# Measures recall based on entities present in the ground truth.
    answer_correctness,   # Measures how close the generated answer is to the ground truth.
    answer_similarity     # Measures the semantic similarity between the generated answer and ground truth.
]

print(f"Running evaluation on: {[m.name for m in REQUESTED_METRICS]}")

Running evaluation on: ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall', 'context_entity_recall', 'answer_correctness', 'answer_similarity']


In [6]:
def evaluate_dataset_iteratively(json_file_path):
    """
    Load data (as a single JSON list or JSONL) and evaluate all samples 
    in the JSON file one by one (iteratively).
    """
    print(f"Loading data from: {json_file_path}")
    
    if not os.path.exists(json_file_path):
        raise FileNotFoundError(f"File not found: {json_file_path}")
    
    all_results_list = []
    data_list = []
    
    # Load data robustly to handle single JSON list or JSONL
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            full_content = f.read().strip()
            
            if not full_content:
                print("File is empty.")
                return None
                
            if full_content.startswith('['):
                data_list = json.loads(full_content)
                print(f"Loaded {len(data_list)} samples from a single JSON list format.")
            else:
                for line in full_content.splitlines():
                    line = line.strip()
                    if line:
                        data_list.append(json.loads(line))
                print(f"Loaded {len(data_list)} samples from JSONL format.")
                
    except json.JSONDecodeError as e:
        print(f"Fatal Error: Could not parse JSON file. Check for formatting errors. {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during file loading: {e}")
        return None

    if not data_list:
        print("No valid data loaded for evaluation.")
        return None
        
    for line_num, raw_data in enumerate(data_list, 1):
        
        if not isinstance(raw_data, dict):
            print(f"Warning: Skipping item {line_num}. Expected dict, got {type(raw_data)}.")
            continue
            
        print(f"\n" + "="*50)
        print(f"Processing Sample {line_num} / {len(data_list)}")
        print("="*50)
        
        try:
            # 2. Transform the single sample into Ragas format
            sample_dict = {}
            sample_dict['question'] = raw_data.get('question', '')
            sample_dict['answer'] = raw_data.get('answer', '')
            
            # Handle context: If it's a list already, use it. If it's a string, wrap it.
            context_data = raw_data.get('context')
            if isinstance(context_data, list):
                sample_dict['contexts'] = context_data
            elif pd.notna(context_data) and context_data is not None:
                # Assuming context is a single string which Ragas expects as a list of strings
                sample_dict['contexts'] = [str(context_data)]
            else:
                 sample_dict['contexts'] = []

            # 'truth' maps to 'reference' (str) and 'ground_truths' (list[str])
            truth_str = raw_data.get('truth', '')
            sample_dict['reference'] = truth_str
            sample_dict['ground_truths'] = [truth_str]
            
            # Create a single-row Dataset
            sample_dataset = Dataset.from_list([sample_dict]) 

            # 3. Run Ragas evaluation on the single sample
            result = evaluate(
                sample_dataset,
                metrics=REQUESTED_METRICS,
                llm=ragas_llm,
                embeddings=ragas_embeddings_llm,
            )
            
            print(f"Sample {line_num} Scores:")
            single_row_scores = result.to_pandas().iloc[0].to_dict()
            for metric in [m.name for m in REQUESTED_METRICS]:
                score = single_row_scores.get(metric)
                if score is not None:
                    print(f"  {metric.replace('_', ' ').title():<25}: {score:.3f}")

            # 4. Store the result 
            all_results_list.append(result.to_pandas())

        except Exception as e:
            question_preview = raw_data.get('question', 'N/A')
            # Print a warning but continue processing other samples
            print(f"Warning: Failed to evaluate sample at index {line_num}: {e}")
            print(f"  Question: {question_preview[:100]}...")
    
    if not all_results_list:
        print("No data processed, skipping evaluation.")
        return None
    
    # 5. Combine all 1-row DataFrames into a single DataFrame
    print("\n" + "="*50)
    print("Combining all results...")
    print("="*50)
    final_results_df = pd.concat(all_results_list, ignore_index=True)
    
    return final_results_df

In [7]:
def save_results(results_df, output_file="evaluation_results.json"):
    """Save detailed results DataFrame to JSON file"""
    if results_df is None: return
    # Filter out non-JSON serializable objects (like list of contexts/ground_truths) for clean export
    export_df = results_df.copy()
    
    # Ragas uses the 'contexts' and 'ground_truths' columns which contain lists/objects.
    records = export_df.to_dict('records')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(records, f, indent=2, ensure_ascii=False)
    print(f"Detailed results saved to {output_file}")

def export_to_csv(results_df, csv_file="evaluation_results.csv"):
    """Export results DataFrame to CSV for easier analysis"""
    if results_df is None: return
    
    # Convert list columns to strings for CSV compatibility
    csv_df = results_df.copy()
    for col in ['contexts', 'ground_truths']:
        if col in csv_df.columns:
            csv_df[col] = csv_df[col].apply(lambda x: '|'.join(x) if isinstance(x, list) else x)
            
    csv_df.to_csv(csv_file, index=False, encoding='utf-8')
    print(f"Results exported to {csv_file}")

def create_summary_statistics(results_df, summary_file="evaluation_summary.json"):
    """Create summary statistics from results DataFrame"""
    if results_df is None: return {}
        
    metric_names = [m.name for m in REQUESTED_METRICS]
    enhanced_summary = {}
    
    for metric_name in metric_names: 
        if metric_name in results_df.columns:
            # Drop NaN scores, which occur if an evaluation fails for a specific metric on a sample
            scores = results_df[metric_name].dropna().tolist() 
            if scores:
                enhanced_summary[metric_name] = {
                    'mean': round(np.mean(scores), 3), 
                    'min': round(min(scores), 3),
                    'max': round(max(scores), 3),
                    'count': len(scores),
                    'std_dev': round(np.std(scores), 3) if len(scores) > 1 else 0
                }
            else:
                enhanced_summary[metric_name] = {'mean': 0, 'count': 0}
    
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(enhanced_summary, f, indent=2, ensure_ascii=False)
    
    print(f"Summary statistics saved to {summary_file}")
    return enhanced_summary 

def print_quick_summary(summary_dict):
    """Print a quick summary (mean scores) to console from the summary dict"""
    if not summary_dict: return
        
    print("\n" + "="*50)
    print("RAGAS QUICK SUMMARY (Mean Scores)")
    print("="*50)
    
    for metric, stats in summary_dict.items():
        print(f"{metric.replace('_', ' ').title():<25}: {stats['mean']:.3f}")
    
    print("="*50)

In [9]:
results_df = None
summary_stats = {}

json_file_path = "2_sample_test_qa_eval.jsonl"

try:
    print(f"Starting iterative evaluation for: {json_file_path}")
    
    results_df = evaluate_dataset_iteratively(json_file_path)
    
    if results_df is not None and not results_df.empty:
        # Save all results
        save_results(results_df)
        export_to_csv(results_df)
        
        # Create and print summary
        summary_stats = create_summary_statistics(results_df)
        print_quick_summary(summary_stats)
        
        print("\n ✅ Evaluation completed successfully!")
        print(f"Total successful samples evaluated: {len(results_df)}")

        print("\nFirst 3 results preview:")
        print("="*80)
        
        # Display the start of the question in the preview
        for i in range(min(3, len(results_df))):
            result = results_df.iloc[i]
            print(f"\nSample {i+1}:")
            
            # Get the question and limit it to the first 100 characters for a preview
            question_preview = result.get('question', 'N/A')
            print(f"Question (Start): {question_preview[:100]}...")
            
            for metric in REQUESTED_METRICS:
                score = result.get(metric.name, 'N/A')
                if isinstance(score, float):
                    score = f"{score:.3f}"
                print(f"{metric.name.replace('_', ' ').title()}: {score}")
            
            print("-" * 40)

    else:
        print("Evaluation did not produce any successful results.")

except FileNotFoundError as e:
    print(f"❌ File not found: {e}")
    print(f"Please check if the file exists at: {json_file_path}")

except RagasException as e:
    print(f"\n❌ [RAGAS ERROR] {e}")
    print("Evaluation halted. Check API keys, endpoints, and rate limits.")
    
except Exception as e:
    print(f"❌ Unexpected error in main execution: {e}")
    print(f"Full traceback:\n{traceback.format_exc()}")

Starting iterative evaluation for: 2_sample_test_qa_eval.jsonl
Loading data from: 2_sample_test_qa_eval.jsonl
Loaded 3 samples from JSONL format.

Processing Sample 1 / 3


Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


Sample 1 Scores:
  Faithfulness             : 0.917
  Answer Relevancy         : 0.975
  Context Precision        : 1.000
  Context Recall           : 1.000
  Context Entity Recall    : 0.500
  Answer Correctness       : 0.329
  Answer Similarity        : 0.964

Processing Sample 2 / 3


Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


Sample 2 Scores:
  Faithfulness             : 1.000
  Answer Relevancy         : 0.974
  Context Precision        : 1.000
  Context Recall           : 1.000
  Context Entity Recall    : 1.000
  Answer Correctness       : 0.370
  Answer Similarity        : 0.959

Processing Sample 3 / 3


Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


Sample 3 Scores:
  Faithfulness             : 1.000
  Answer Relevancy         : 0.955
  Context Precision        : 1.000
  Context Recall           : 1.000
  Context Entity Recall    : 1.000
  Answer Correctness       : 0.579
  Answer Similarity        : 0.983

Combining all results...
Detailed results saved to evaluation_results.json
Results exported to evaluation_results.csv
Summary statistics saved to evaluation_summary.json

RAGAS QUICK SUMMARY (Mean Scores)
Faithfulness             : 0.972
Answer Relevancy         : 0.968
Context Precision        : 1.000
Context Recall           : 1.000
Context Entity Recall    : 0.833
Answer Correctness       : 0.426
Answer Similarity        : 0.969

 ✅ Evaluation completed successfully!
Total successful samples evaluated: 3

First 3 results preview:

Sample 1:
Question (Start): N/A...
Faithfulness: 0.917
Answer Relevancy: 0.975
Context Precision: 1.000
Context Recall: 1.000
Context Entity Recall: 0.500
Answer Correctness: 0.329
Answer Similarit

In [16]:
# --- This is the main execution block ---
# (Note: We remove the 'if __name__ == "__main__":' check for notebooks)

results_df = None
summary_stats = {}

# ⚠️ Set the path to your data file
# This path was from your original script. Update it if needed.
json_file_path = "200_sample_test_qa_eval.jsonl"

try:
    print(f"Starting iterative evaluation for: {json_file_path}")
    
    results_df = evaluate_dataset_iteratively(json_file_path)
    
    if results_df is not None and not results_df.empty:
        # Save all results
        save_results(results_df)
        export_to_csv(results_df)
        
        # Create and print summary
        summary_stats = create_summary_statistics(results_df)
        print_quick_summary(summary_stats)
        
        print("\n ✅ Evaluation completed successfully!")
        print(f"Total successful samples evaluated: {len(results_df)}")

        print("\nFirst 3 results preview:")
        print("="*80)
        
        # Display the start of the question in the preview
        for i in range(min(3, len(results_df))):
            result = results_df.iloc[i]
            print(f"\nSample {i+1}:")
            
            # Get the question and limit it to the first 100 characters for a preview
            question_preview = result.get('question', 'N/A')
            print(f"Question (Start): {question_preview[:100]}...")
            
            for metric in REQUESTED_METRICS:
                score = result.get(metric.name, 'N/A')
                if isinstance(score, float):
                    score = f"{score:.3f}"
                print(f"{metric.name.replace('_', ' ').title()}: {score}")
            
            print("-" * 40)

    else:
        print("Evaluation did not produce any successful results.")

except FileNotFoundError as e:
    print(f"❌ File not found: {e}")
    print(f"Please check if the file exists at: {json_file_path}")

except RagasException as e:
    print(f"\n❌ [RAGAS ERROR] {e}")
    print("Evaluation halted. Check API keys, endpoints, and rate limits.")
    
except Exception as e:
    print(f"❌ Unexpected error in main execution: {e}")
    print(f"Full traceback:\n{traceback.format_exc()}")

Starting iterative evaluation for: 200_sample_test_qa_eval.jsonl
Loading data from: 200_sample_test_qa_eval.jsonl
Loaded 200 samples from JSONL format.

Processing Sample 1 / 200


Evaluating: 100%|██████████| 7/7 [00:45<00:00,  6.57s/it]


Sample 1 Scores:
  Faithfulness             : 0.923
  Answer Relevancy         : 0.975
  Context Precision        : 1.000
  Context Recall           : 1.000
  Context Entity Recall    : 0.333
  Answer Correctness       : 0.429
  Answer Similarity        : 0.964

Processing Sample 2 / 200


Evaluating: 100%|██████████| 7/7 [00:48<00:00,  6.91s/it]


Sample 2 Scores:
  Faithfulness             : 1.000
  Answer Relevancy         : 0.983
  Context Precision        : 1.000
  Context Recall           : 1.000
  Context Entity Recall    : 0.667
  Answer Correctness       : 0.311
  Answer Similarity        : 0.959

Processing Sample 3 / 200


...

Sample 198 Scores:
  Faithfulness             : 1.000
  Answer Relevancy         : 0.972
  Context Precision        : 1.000
  Context Recall           : 0.500
  Context Entity Recall    : 1.000
  Answer Correctness       : 0.426
  Answer Similarity        : 0.954

Processing Sample 199 / 200


Evaluating: 100%|██████████| 7/7 [00:38<00:00,  5.54s/it]


Sample 199 Scores:
  Faithfulness             : 1.000
  Answer Relevancy         : 0.984
  Context Precision        : 1.000
  Context Recall           : 1.000
  Context Entity Recall    : 1.000
  Answer Correctness       : 0.642
  Answer Similarity        : 0.968

Processing Sample 200 / 200


Evaluating: 100%|██████████| 7/7 [00:34<00:00,  4.93s/it]


Sample 200 Scores:
  Faithfulness             : 1.000
  Answer Relevancy         : 0.990
  Context Precision        : 1.000
  Context Recall           : 1.000
  Context Entity Recall    : 1.000
  Answer Correctness       : 0.315
  Answer Similarity        : 0.946

Combining all results...
Detailed results saved to evaluation_results.json
Results exported to evaluation_results.csv
Summary statistics saved to evaluation_summary.json

RAGAS QUICK SUMMARY (Mean Scores)
Faithfulness             : 0.926
Answer Relevancy         : 0.921
Context Precision        : 0.867
Context Recall           : 0.687
Context Entity Recall    : 0.423
Answer Correctness       : 0.452
Answer Similarity        : 0.939

 ✅ Evaluation completed successfully!
Total successful samples evaluated: 200

First 3 results preview:

Sample 1:
Question (Start): N/A...
Faithfulness: 0.923
Answer Relevancy: 0.975
Context Precision: 1.000
Context Recall: 1.000
Context Entity Recall: 0.333
Answer Correctness: 0.429
Answer Simil